In [1]:
import numpy as np
import pandas as pd
import jieba
import re
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

## clean data

In [5]:
movie_corpus= pd.read_csv('movie_comments.csv',encoding='utf8')

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [622]:
movie_corpus.head(30)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2
2,3,https://movie.douban.com/subject/26363254/,战狼2,吴京的炒作水平不输冯小刚，但小刚至少不会用主旋律来炒作…吴京让人看了不舒服，为了主旋律而主旋...,2
3,4,https://movie.douban.com/subject/26363254/,战狼2,凭良心说，好看到不像《战狼1》的续集，完虐《湄公河行动》。,4
4,5,https://movie.douban.com/subject/26363254/,战狼2,中二得很,1
5,6,https://movie.douban.com/subject/26363254/,战狼2,“犯我中华者，虽远必诛”，吴京比这句话还要意淫一百倍。,1
6,7,https://movie.douban.com/subject/26363254/,战狼2,脑子是个好东西，希望编剧们都能有。,2
7,8,https://movie.douban.com/subject/26363254/,战狼2,三星半，实打实的7分。第一集在爱国主旋律内部做着各种置换与较劲，但第二集才真正显露吴京的野心...,4
8,9,https://movie.douban.com/subject/26363254/,战狼2,开篇长镜头惊险大气引人入胜 结合了水平不俗的快剪下实打实的真刀真枪 让人不禁热血沸腾 特别弹...,4
9,10,https://movie.douban.com/subject/26363254/,战狼2,15/100吴京的冷峰在这部里即像成龙，又像杰森斯坦森，但体制外的同类型电影，主角总是代表个...,1


In [22]:
XY=movie_corpus.iloc[:,3:].copy()

In [23]:
cleaned_XY=a.dropna(subset=['comment'])

In [24]:
pattern = re.compile('\w+')

In [32]:
def data_clean(string):
    d=''.join(pattern.findall(string))
    s=' '.join(jieba.cut(d))
    return s

In [42]:
a=cleaned_XY['comment']
cleaned_XY['cleaned_comment'] = a.apply(data_clean)

In [45]:
cleaned_XY.drop(labels='comment',axis=1,inplace=True)

In [53]:
cleaned_XY.rename(columns={'star':'Y','cleaned_comment':'X'},inplace=True)

In [54]:
cleaned_XY.head(10)

,Y,X
0,1,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐
1,2,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...
2,2,吴京 的 炒作 水平 不输 冯小刚 但小刚 至少 不会 用 主旋律 来 炒作 吴京 让 人 ...
3,4,凭良心说 好 看到 不像 战狼 1 的 续集 完虐 湄公河 行动
4,1,中二得 很
5,1,犯 我 中华 者 虽远必 诛 吴京 比 这句 话 还要 意淫 一百倍
6,2,脑子 是 个 好 东西 希望 编剧 们 都 能 有
7,4,三星 半 实打实 的 7 分 第一集 在 爱国 主旋律 内部 做 着 各种 置换 与 较劲 ...
8,4,开篇 长镜头 惊险 大气 引人入胜 结合 了 水平 不俗 的 快 剪下 实打实 的 真刀真枪...
9,1,15100 吴京 的 冷峰 在 这部 里 即 像 成龙 又 像杰 森斯坦 森但 体制 外 的...


In [55]:
cleaned_XY.shape

(261495, 2)

In [56]:
cleaned_XY.to_csv('cleaned_for_train.csv')

## train word_vector

In [60]:
with open('cleaned_for_train.csv','r',encoding='utf8') as f:
    word_model = Word2Vec(LineSentence(f),size=100)

In [68]:
word_model.save('news_vector')

In [73]:
a = Word2Vec.load('news_vector')

In [74]:
a=word_model.wv
a.most_similar('我')

[('你', 0.6124907732009888),
 ('俺', 0.5461215972900391),
 ('我会', 0.536949872970581),
 ('大家', 0.5187957286834717),
 ('她', 0.5101457834243774),
 ('当初', 0.5062898397445679),
 ('人', 0.4970146417617798),
 ('观众', 0.4938845634460449),
 ('一直', 0.4937843382358551),
 ('它', 0.48911339044570923)]

## sentence vector

In [78]:
line=jieba.cut('今天天气很好我们一起去郊游吧')

In [82]:
cleaned_XY.rename(columns={'X':'Sen'},inplace=True)

In [83]:
cleaned_XY.head(2)

,Y,Sen
0,1,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐
1,2,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...


In [86]:
model = Word2Vec.load('news_vector')
wv = model.wv

In [586]:
def get_sentence_vector(sentence):
    ws = sentence.split()
    ws = [ a for a in ws if a in wv.vocab]
    matrix = np.sum(np.asarray([wv[a] for a in ws ]),axis=0)/ len(ws)
    return matrix

In [587]:
cleaned_XY['X'] =cleaned_XY['Sen'].apply(get_sentence_vector)

In [588]:
cleaned_XY.shape

(253677, 3)

In [589]:
cleaned_XY.head(2)

,Y,Sen,X
0,1,吴京 意淫 到 了 脑残 的 地步 看 了 恶心 想 吐,"[-0.75965554, 0.585755, 0.653013, 0.5308875, -..."
1,2,首映礼 看 的 太 恐怖 了 这个 电影 不讲道理 的 完全 就是 吴京 在 实现 他 这个...,"[-0.8425579, 0.041578423, 0.37888587, 0.213676..."


In [590]:
cleaned_XY.dropna(subset=['X'],inplace=True) #remove na

In [591]:
Y = np.asarray(cleaned_XY['Y'])

In [592]:
X = np.concatenate([a.reshape(1,-1) for a in cleaned_XY['X']])

In [593]:
X.shape

(253677, 100)

In [594]:
Y.shape

(253677,)

## data split

In [599]:
X.dtype

dtype('float32')

In [600]:
Y=Y.astype('float32') #type convert

In [601]:
Y.dtype

dtype('float32')

In [602]:
X.dtype

dtype('float32')

In [630]:
data = np.concatenate([X, Y.reshape(-1,1)],axis=1)

AttributeError: 'Tensor' object has no attribute 'reshape'

In [604]:
data.shape

(253677, 101)

In [605]:
data.dtype

dtype('float32')

In [606]:
data[0]

array([-0.75965554,  0.585755  ,  0.653013  ,  0.5308875 , -0.8291249 ,
       -0.1277238 , -0.26111695,  0.42433444,  0.01836779, -0.43668666,
       -0.25576475, -0.18091045,  0.2649455 , -0.49695602,  1.291426  ,
       -0.78607655, -0.36882496, -0.45865402, -0.82139283,  0.1488651 ,
        0.08990908,  0.45723435,  0.07786971,  0.23718983,  0.5163208 ,
       -0.04585673,  0.69643956,  0.1286376 ,  0.23877235, -0.03645267,
        0.43516123, -0.04573588,  0.06817112, -0.54541665,  0.10132736,
       -0.10005027,  0.4477826 , -0.16605018, -0.05341306,  0.01387484,
       -0.2994766 , -0.01277458,  0.7380431 ,  0.4772904 ,  0.18722458,
       -0.01419878,  0.5047675 ,  0.30156338,  0.5422192 ,  0.03767951,
        0.788621  , -0.0073684 ,  0.20435174,  0.6590789 , -0.7590256 ,
       -0.42855465,  0.6293128 , -0.8665336 , -0.24041514,  0.2800518 ,
       -0.84311867,  0.6918037 ,  0.17803045,  0.55546576,  0.31053945,
       -0.66440886,  0.03638462, -0.36203524, -0.02935234,  0.40

In [607]:
np.random.shuffle(data) #shuffer

In [608]:
n_train=np.floor(data.shape[0] * 0.6).astype('int')
n_valid=np.floor(data.shape[0] * 0.2).astype('int')
n_test=np.floor(data.shape[0] - n_train - n_valid).astype('int')

In [609]:
data.shape

(253677, 101)

In [610]:
def get_y_vector(y,c):
    r = np.zeros([len(y),c])
    for i, j in enumerate(y): r[i,j.astype('int') -1]=1
    return r

In [611]:
train_x = data[:n_train,:-1]
train_y = get_y_vector(data[:n_train,-1],5)
valid_x = data[:n_valid,:-1]
valid_y = get_y_vector(data[:n_valid,-1],5) 
test_x = data[:n_test,:-1]
test_y = get_y_vector(data[:n_test,-1],5)  

In [612]:
train_x.shape

(152206, 100)

In [613]:
valid_x.dtype

dtype('float32')

In [614]:
train_y[:10]

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.]])

In [615]:
cosine_dis(train_x[6],train_x[0])

0.632558

In [616]:
def cosine_dis(vector1, vector2):
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*(np.linalg.norm(vector2)))

In [617]:
train_y.shape

(152206, 5)

## train network

In [625]:
train_x.shape

(152206, 100)

In [626]:
train_y.shape

(152206, 5)

In [629]:
train_y[20]

array([0., 1., 0., 0., 0.])

In [663]:
X = tf.placeholder(tf.float32,[None, 100])
Y = tf.placeholder(tf.float32,[None, 5])
W0 = tf.Variable(tf.truncated_normal([100,300]))
b0 = tf.Variable(tf.zeros([300]))
W1 = tf.Variable(tf.truncated_normal([300,5]))
b1 = tf.Variable(tf.zeros([5]))
z0 = tf.matmul(X, W0) + b0
z0 = tf.sigmoid(z0)
# z0 = tf.nn.relu(z0)
z1 = tf.matmul(z0, W1) + b1
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z1, labels=tf.cast(Y, dtype=tf.int32)))
optimizer = tf.train.GradientDescentOptimizer(5)
optimizer = optimizer.minimize(loss)
correct_pred = tf.equal(tf.argmax(z1, 1), tf.argmax(Y, 1))
accuracy = 100*tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

In [633]:
train_x.shape

(152206, 100)

In [665]:
with tf.Session() as S:
    S.run(init)
    for i in range(2000):
        for j in range(500):
            XX = train_x[j*300:(j+1)*300]
            YY = train_y[j*300:(j+1)*300]
            
            S.run(optimizer,feed_dict={X:XX,Y:YY})
        if i%5 ==0:
            print('train acc:',S.run(accuracy,feed_dict={X: train_x,Y: train_y}))
            print('test acc:',S.run(accuracy,feed_dict={X: test_x,Y: test_y}))

train acc: 37.105633
test acc: 36.845634
train acc: 48.47312
test acc: 47.948204
train acc: 50.733875
test acc: 50.208927
train acc: 51.814644
test acc: 51.168793
train acc: 52.476906
test acc: 51.902
train acc: 52.907246
test acc: 52.300144
train acc: 53.33167
test acc: 52.7377
train acc: 53.679222
test acc: 53.143723
train acc: 54.05831
test acc: 53.553688
train acc: 54.282356
test acc: 53.802032
train acc: 54.51888
test acc: 54.03067
train acc: 54.733063
test acc: 54.28296
train acc: 54.957752
test acc: 54.485966
train acc: 55.09507
test acc: 54.57269
train acc: 55.27706
test acc: 54.755993
train acc: 55.426853
test acc: 54.92944
train acc: 55.53592
test acc: 55.071346
train acc: 55.683743
test acc: 55.225086
train acc: 55.784267
test acc: 55.307873
train acc: 55.847996
test acc: 55.374886


## test tfidf vector

In [424]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [426]:
movie_corpus['comment'].head(2)

0                                   吴京意淫到了脑残的地步，看了恶心想吐
1    首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...
Name: comment, dtype: object

In [448]:
tfidf_v = TfidfVectorizer(max_features=2000)

In [435]:
td_doc = movie_corpus.copy()

In [436]:
td_doc.head(2)

,id,link,name,comment,star
0,1,https://movie.douban.com/subject/26363254/,战狼2,吴京意淫到了脑残的地步，看了恶心想吐,1
1,2,https://movie.douban.com/subject/26363254/,战狼2,首映礼看的。太恐怖了这个电影，不讲道理的，完全就是吴京在实现他这个小粉红的英雄梦。各种装备轮...,2


In [438]:
td_doc.dropna(subset=['comment','star'],axis=0,inplace=True)

In [449]:
a=tfidf_v.fit_transform(td_doc['comment'])

In [450]:
tfidf_array=a.toarray()

In [451]:
tfidf_array.shape

(261495, 2000)

In [498]:
td_doc['star'][568]=2

In [507]:
def get_y_vector1(y,c):
    r = np.zeros([len(y),c])
    for i, j in enumerate(y): r[i,j -1]=1
    return r

In [671]:
X1 = tfidf_array
Y1 = get_y_vector1(td_doc['star'].astype('int'),5)

In [672]:
X1.shape

(261495, 2000)

In [673]:
Y1.shape

(261495, 5)

In [677]:
X = tf.placeholder(tf.float32,[None, 2000])
Y = tf.placeholder(tf.float32,[None, 5])
W0 = tf.Variable(tf.truncated_normal([2000,50]))
b0 = tf.Variable(tf.zeros([50]))
W1 = tf.Variable(tf.truncated_normal([50,5]))
b1 = tf.Variable(tf.zeros([5]))
z0 = tf.matmul(X, W0) + b0
z0 = tf.sigmoid(z0)
# z0 = tf.nn.relu(z0)
z1 = tf.matmul(z0, W1) + b1
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=z1, labels=tf.cast(Y, dtype=tf.int32)))
optimizer = tf.train.GradientDescentOptimizer(50)
optimizer = optimizer.minimize(loss)
correct_pred = tf.equal(tf.argmax(z1, 1), tf.argmax(Y, 1))
accuracy = 100*tf.reduce_mean(tf.cast(correct_pred, tf.float32))
init = tf.global_variables_initializer()

In [675]:
X1.shape

(261495, 2000)

In [678]:
with tf.Session() as S:
    S.run(init)
    for i in range(10):
        for j in range(650):
            XX = X1[j*300:(j+1)*300]
            YY = Y1[j*300:(j+1)*300]
            
            S.run(optimizer,feed_dict={X:XX,Y:YY})
        if i%2 ==0:
            print('train acc:',S.run(accuracy,feed_dict={X: X1[:190000] ,Y: Y1[:190000]}))
            print('test acc:',S.run(accuracy,feed_dict={X: X1[190000:],Y: Y1[190000:]}))

train acc: 23.782106
test acc: 28.653753
train acc: 23.782106
test acc: 28.653753
train acc: 24.54842
test acc: 17.725716
train acc: 32.25
test acc: 31.51689
train acc: 23.782106
test acc: 28.653753
